# **Predict health cost**
*En esta notebook se realizo un modelo de Tensorflow para predicir el costo del servicio para una aseguradora*

Importamos las librerias necesarias para el proyecto

In [1]:
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
try:
  # unicamente correr este codigo en Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.model_selection import train_test_split

Importamos los datos y generamos el **Dataset**

In [2]:
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

--2022-06-18 03:17:13--  https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50264 (49K) [text/csv]
Saving to: ‘insurance.csv’

insurance.csv       100%[===================>]  49.09K  --.-KB/s    in 0s      

2022-06-18 03:17:13 (151 MB/s) - ‘insurance.csv’ saved [50264/50264]



,age,sex,bmi,children,smoker,region,expenses
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95
1337,61,female,29.1,0,yes,northwest,29141.36


transformamos datos categoricos a numericos con Feature engenier, el metodo utilizado fue **get_dummies**

In [3]:
dms = pd.get_dummies(dataset[["sex","smoker","region"]])
y = dataset["expenses"]
dataset.drop(["sex","smoker","region"],axis=1)



,age,bmi,children,expenses
0,19,27.9,0,16884.92
1,18,33.8,1,1725.55
2,28,33.0,3,4449.46
3,33,22.7,0,21984.47
4,32,28.9,0,3866.86
...,...,...,...,...
1333,50,31.0,3,10600.55
1334,18,31.9,0,2205.98
1335,18,36.9,0,1629.83
1336,21,25.8,0,2007.95


Lo unimos al Dataset

In [4]:
X = pd.concat([dataset,dms[["sex_female","sex_male","smoker_no","smoker_yes","region_northeast","region_northwest","region_southeast","region_southwest"]]],axis=1)
X

,age,sex,bmi,children,smoker,region,expenses,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,female,27.9,0,yes,southwest,16884.92,1,0,0,1,0,0,0,1
1,18,male,33.8,1,no,southeast,1725.55,0,1,1,0,0,0,1,0
2,28,male,33.0,3,no,southeast,4449.46,0,1,1,0,0,0,1,0
3,33,male,22.7,0,no,northwest,21984.47,0,1,1,0,0,1,0,0
4,32,male,28.9,0,no,northwest,3866.86,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55,0,1,1,0,0,1,0,0
1334,18,female,31.9,0,no,northeast,2205.98,1,0,1,0,1,0,0,0
1335,18,female,36.9,0,no,southeast,1629.83,1,0,1,0,0,0,1,0
1336,21,female,25.8,0,no,southwest,2007.95,1,0,1,0,0,0,0,1


In [5]:
X = X.drop(["sex","smoker","region"],axis=1)

**Hora de hacer el split**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [7]:
X_test = X_test.drop(["expenses"],axis=1)
X_train = X_train.drop(["expenses"],axis=1)

# Modelo
En este caso vamos sequential de Keras, El modelo secuencial es una pila lineal de capas

In [14]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 1))

Compilamos el modelo

In [16]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

# Momento de entrenarlo

In [17]:
model.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose=2)

Epoch 1/100
107/107 - 2s - loss: 205862.7812 - acc: 0.0000e+00 - 2s/epoch - 15ms/step
Epoch 2/100
107/107 - 0s - loss: 205862.7188 - acc: 0.0000e+00 - 225ms/epoch - 2ms/step
Epoch 3/100
107/107 - 0s - loss: 205862.7188 - acc: 0.0000e+00 - 221ms/epoch - 2ms/step
Epoch 4/100
107/107 - 0s - loss: 205862.7500 - acc: 0.0000e+00 - 229ms/epoch - 2ms/step
Epoch 5/100
107/107 - 0s - loss: 205862.8125 - acc: 0.0000e+00 - 221ms/epoch - 2ms/step
Epoch 6/100
107/107 - 0s - loss: 205862.7812 - acc: 0.0000e+00 - 222ms/epoch - 2ms/step
Epoch 7/100
107/107 - 0s - loss: 205862.7188 - acc: 0.0000e+00 - 221ms/epoch - 2ms/step
Epoch 8/100
107/107 - 0s - loss: 205862.7188 - acc: 0.0000e+00 - 229ms/epoch - 2ms/step
Epoch 9/100
107/107 - 0s - loss: 205862.7656 - acc: 0.0000e+00 - 221ms/epoch - 2ms/step
Epoch 10/100
107/107 - 0s - loss: 205862.7812 - acc: 0.0000e+00 - 229ms/epoch - 2ms/step
Epoch 11/100
107/107 - 0s - loss: 205862.7500 - acc: 0.0000e+00 - 218ms/epoch - 2ms/step
Epoch 12/100
107/107 - 0s - loss

In [27]:
mae = model.evaluate(X_test, y_test, verbose=2)
print(mae)





9/9 - 0s - loss: 200035.6406 - acc: 0.0000e+00 - 58ms/epoch - 6ms/step
[200035.640625, 0.0]


#  El modelo ya se encuentra listo para predecir 

Aca vemos a quien vamos a predecir el costo del seguro de vida

In [26]:
print(X_test[:5])

      age   bmi  children  sex_female  sex_male  smoker_no  smoker_yes  \
764    45  25.2         2           1         0          1           0   
887    36  30.0         0           1         0          1           0   
890    64  26.9         0           1         0          0           1   
1293   46  25.7         3           0         1          1           0   
259    19  31.9         0           0         1          0           1   

      region_northeast  region_northwest  region_southeast  region_southwest  
764                  1                 0                 0                 0  
887                  0                 1                 0                 0  
890                  0                 1                 0                 0  
1293                 0                 1                 0                 0  
259                  0                 1                 0                 0  


# **Resultado del modelo prediciendo el valor del seguro de vida**

In [18]:
model.predict(X_test[:5])

array([[-2.316605 ],
       [-1.145263 ],
       [-4.7665806],
       [-2.6956413],
       [-1.2092626]], dtype=float32)

😲